In [157]:
# !pip install numpy
# ! pip install deap

# The class for reading the knapsack dataset file.

In [158]:
import pandas as pd
import numpy as np
from copy import deepcopy
from distutils.command.build_scripts import first_line_re
from tkinter.tix import COLUMN
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque

class Knapsack:
    '''For each instance, we have following fields: 
            the 1st line of the  file contains the number of items N and the knapsack capacity C.,
            2nd onwards contains the Value and Weight of each item, separated by a space.
    '''
    # main constructor 
    def __init__(self, M, capacity, items):
        '''N is the number of items, i.e. len(items)'''
        self.M = M
        self.capacity = capacity
        self.items = items
        self.items_dict  = {}
        i = 0
        for x in self.items.to_records(index=False):
            self.items_dict[i] = tuple(x)
            i += 1
        
    @classmethod
    def constructFromFile(cls, filePath):
        '''Read from file and construct an instance of Knapsack'''
        with open(filePath, 'r') as file:
            first_line = file.readline()
            N,capacity = int(first_line.split(' ')[0]),int(first_line.split(' ')[1])
            # first_line = first_line.split(' ')
        itemsdf = pd.read_csv(filePath,delim_whitespace=True, skiprows=range(1), header=None)
        itemsdf.columns = ['Value', 'Weight']
        
        # add a index column for the items
        # items.index = items.index + 1
        return cls(N, capacity, itemsdf)
    
    def __str__(self):
        return 'N = {}, capacity = {}, \nitems =\n {}'.format(self.M, self.capacity, self.items)    
ds_10_269 = Knapsack.constructFromFile('10_269')
# print(ten_269)
# len(ds_10_269.items)
    

# part 1 :

Develop a GA to solve the 0-1 knapsack problem and apply it to the provided three instances. You can use a library or write the program from scratch (GA is not hard to code). 
You should 

- Determine the proper individual representation and explain the reasons. 
- Design the proper fitness function and justify it. 
- Design proper crossover and mutation operators. 
- Set proper GA parameter values, such as population size, crossover and mutation rates, selection scheme. 
- For each instance, run the GA for 5 times with different random seeds. Present the mean and standard deviation of the GA in the 5 runs. 
- Compare the results with the optimal values, and make discussions. 
- Draw the convergence curve of the GA for each instance (x-axis being the number of generations, y-axis the average fitness of the 5 best solutions in the population of the xth generation from the 5 runs). Make discussions about the convergence curve and draw your conclusions.

In [159]:
from asyncio import constants
from json import tool
from deap import creator, base, gp, tools, algorithms # core functionality of DEAP
import array
import random
import json
import math # for checking the fitness of an individual, i.e. math.isinf(weight)
import matplotlib.pyplot as plt
# Python internal operators for object comparisons, 
# logical comparisons, arithmetic operations, and sequence operations
import operator 


# creator is  usually used to define the type of the individual and fitness classes

# goal:to maximize the value and do not exceed the capacity of the knapsack
# define strategies with different priorities for optimizing multiple goals by using FitnessCompound
# 1 for maximize value, -1 for minimize weight, 
# creator.create("FitnessCompound", base.Fitness, weights=(1.0,-1.0)) 

# according to slide, fitness value has been reduced to 1 dimension, so just use FitnessMax
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Individual should be a list of binary values, i.e. a list of 0s and 1s
creator.create("Individual", list, fitness=creator.FitnessMax)


In [160]:
def evaluateFunction(ds:Knapsack,individual:creator.Individual,  penality_coef:float=999999.0 ,): 
    #FIXME: this is not the correct way to evaluate the individual
    """fitness evaluation function for the knapsack problem.
    It is inspired by the slide and this paper
    :https://www.dataminingapps.com/2017/03/solving-the-knapsack-problem-with-a-simple-genetic-algorithm/ 

    Args:
        ds (Knapsack): for calculating value and weight
        individual (creator.Individual): a list of binary values, i.e. a list of 0s and 1s
        penality_coef (float, optional): 
                        Very large value: always ignore infeasible solutions 
                        Zero: only consider quality (bad choice, will get 1111....1) 
                        Somewhere in between, parameter tuning, or adaptively change 𝛼

    Returns:
        tuple: the fitness of the individual
    """
    value, weight = 0.0, 0.0
    for i in range(len(individual)):
        value += ds.items.iloc[i]['Value'] * individual[i]
        weight += ds.items.iloc[i]['Weight'] * individual[i]
    # all_item_weight = ds.items['Weight'].sum()
    # penalty = abs(weight - ds.capacity) * all_item_weight
    penalty = penality_coef*max(0, weight - ds.capacity)
    fitnessVal = value-penalty
    
    return fitnessVal,

    # if weight > ds.capacity:
    #     return -math.inf, math.inf # ensure overweighted bags are dominated
    # return value, weight 

def calculate_weight_values(ds:Knapsack, individual:creator.Individual):
    value, weight = 0.0,0.0
    for i in range(len(individual)):
        value += ds.items.iloc[i]['Value'] * individual[i]
        weight += ds.items.iloc[i]['Weight'] * individual[i]

    # print(f"Individual: {individual}\nTotal value: {value} \n Total weight: {weight}")
    return {"value":value, "weight":weight, "individual_list":individual}
    # return weight

In [161]:

# define some constants for the genetic algorithm
CONSTANTS_DICT = {
    "POPULATION_SIZE": 100, # number of individuals in each population
    "MAX_GENERATIONS": 200, # number of generations to run the algorithm
    "CROSSOVER_RATE": 1.0, # crossover rate should always be 100%, based on slides
    "MUTATION_RATE": 0.1, # mutation rate
    "ELITIST_PERCENTAGE": 0.05, # percentage of the best individuals to keep in the next generation
    
}


In [162]:

# toolbox is a class contains the operators that we will use in our genetic programming algorithm
# it can be also be used as the container of methods which enables us to add new methods to the toolbox 
def setup_toolbox(ds:Knapsack,randSeed:int=12) -> base.Toolbox:
    toolbox = base.Toolbox()
    # for population size, we use the random.randint function to generate a random integer in the range [min, max]
    random.seed(randSeed)
    toolbox.register("attr_bool", random.randint, 0, 1) # register a method to generate random boolean values
    toolbox.register("IndividualCreator", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(ds.items)) # register a method to generate random individuals
    
    # N is not specificied, so need to specify number of individuals to generate within each population when we call it later
    toolbox.register("PopulationCreator", tools.initRepeat, list, toolbox.IndividualCreator) 
    
    # multi-objective problem, we have selected the NSGA-II selection scheme
    toolbox.register("elitism", tools.selBest, k=int(CONSTANTS_DICT["ELITIST_PERCENTAGE"]*ds.M))
    toolbox.register("select", tools.selTournament, k=2, tournsize=3)
    
    toolbox.register("mate", tools.cxOnePoint) # TODO: might need to change this to cxOnePoint
    # indpb refer to the probability of mutate happening on each gene, it is NOT the same as mutation rate
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ds.M) # TODO: might need to change this to mutUniformInt
    # local search operator
    # toolbox.register("local_search", algorithms.)
    
    
    # register the evaluation function
    toolbox.register("evaluate", evaluateFunction,ds) # register a method to evaluate the fitness of an individual
    return toolbox

For GA framework implementation

In [163]:
import copy
from select import select


def run_GA_framework(ds:Knapsack, randSeed:int=12) -> creator.Individual:
    '''
    Run the genetic algorithm framework
    '''
    # for toolbox
    random.seed(randSeed)
    toolbox = setup_toolbox(ds,randSeed)
    # for record keeping
    logbook = tools.Logbook()    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis = 0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    # stats.register("bestIndividualValue", calculate_weight_values, ds)
    
    
    # create the initial population
    population = toolbox.PopulationCreator(n=CONSTANTS_DICT["POPULATION_SIZE"])
    # evaluate the fitness of the initial population, and assign the fitness to each individual
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    
    
    # start the evolution
    best_feasible_individual = None
    for gen_counter in range(CONSTANTS_DICT["MAX_GENERATIONS"]):
        best_feasible_individual = tools.selBest(population, 1)[0]
        
        # stats.best(ds,best_feasible_individual)
        
        temp = calculate_weight_values(ds,best_feasible_individual)
        # record the statistics of the current generation
        record = stats.compile(population) #calculate_weight_values, best_feasible_individual,ds )
        logbook.record(gen=gen_counter, bestIndividualValue=temp["value"], bestIndividualWeight=temp["weight"], bestIndividualList=temp["individual_list"],**record, **temp)
        # append temp to logbook
        # logbook.record(gen=gen_counter, **temp)
        # print(logbook)
        
        
        # apply elitism to obtain the best individuals in the current generation
        offspring = toolbox.elitism(population)

        # repeat until the offspring has the same size as the population
        while len(offspring) < CONSTANTS_DICT["POPULATION_SIZE"]:
            # apply selection
            parent1,parent2 = toolbox.select(population)

            # apply crossover
            c1,c2 = toolbox.mate(copy.deepcopy(parent1),copy.deepcopy(parent2))
            
            # apply mutation to the children
            for child in [c1,c2]:
                if random.random() < CONSTANTS_DICT["MUTATION_RATE"]:
                    toolbox.mutate(child)
                    del child.fitness.values
            # TODO: apply local search to the children
            
            
            # get and assign the fitness of the children
            c1.fitness.values = toolbox.evaluate(c1)
            c2.fitness.values = toolbox.evaluate(c2)
            offspring.append(c1)
            offspring.append(c2)
        # replace the current population with the offspring new gwneration
        population[:] = offspring

    return best_feasible_individual, logbook, stats


In [164]:
ds_10_269 = Knapsack.constructFromFile('./10_269')
best_feasible_individual,logbook,stats = run_GA_framework(ds_10_269)
logbook.header = "gen", "avg", "std", "min", "max", "value", "weight", "individual_list"
print(logbook)
type(logbook)

gen	avg           	std                	min              	max   	value	weight	individual_list               
0  	[-45909742.85]	[60741148.11008846]	[-2.65999332e+08]	[269.]	269  	243   	[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]
1  	[-11529804.14]	[26988241.31174806]	[-1.50999503e+08]	[274.]	274  	266   	[1, 0, 1, 0, 0, 0, 0, 0, 1, 1]
2  	[-5529802.4]  	[18186467.7730796] 	[-1.2099957e+08] 	[273.]	273  	266   	[0, 0, 1, 0, 1, 1, 0, 0, 1, 1]
3  	[-5689779.09] 	[20734312.96234656]	[-1.30999484e+08]	[283.]	283  	245   	[0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
4  	[-5779759.67] 	[16373461.79394905]	[-91999580.]     	[293.]	293  	249   	[0, 1, 0, 0, 0, 1, 0, 1, 1, 1]
5  	[-3289744.88] 	[11050122.21614455]	[-58999607.]     	[293.]	293  	249   	[0, 1, 0, 0, 0, 1, 0, 1, 1, 1]
6  	[-3399730.51] 	[14221089.47348857]	[-91999580.]     	[279.]	279  	247   	[0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
7  	[-829726.77]  	[5323621.53823876] 	[-39999620.]     	[284.]	284  	256   	[0, 0, 1, 0, 1, 0, 0, 1, 1, 1]
8  	[-1599722.18] 	[9421237.

deap.tools.support.Logbook

In [165]:
assert False
# calculate_weight_values(ds_10_269,best_feasible_individual)
best_feasible_individual

AssertionError: 

In [ ]:
ds_100_95 = Knapsack.constructFromFile("./100_995")
best_feasible_individual = run_GA_framework(ds_100_95)
calculate_weight_values(best_feasible_individual,ds_100_95)

KeyboardInterrupt: 

In [ ]:
ds_23_10000 = Knapsack.constructFromFile("./23_10000")
best_feasible_individual = run_GA_framework(ds_23_10000)

In [ ]:
calculate_weight_values(best_feasible_individual,ds_23_10000)